# Randomized Benchmarking: Unitarity RB

In [ ]:
# See rb_functions in examples folder for related usage
# Needs in terminal:
# $ quilc -S
# $ qvm -S

from forest.benchmarking.analysis.fitting import make_figure
from forest.benchmarking.randomized_benchmarking import *
from forest.benchmarking.stratified_experiment import StratifiedExperiment
from pyquil.api import get_benchmarker
import numpy as np
from typing import List, Tuple, Callable
from pandas import DataFrame, Series
from pyquil import Program, get_qc
from pyquil import noise

# Add functionality to inject understandable error into RB sequence

In [ ]:
# similar methods found in tests/test_rb.py
def insert_noise(programs: List[Program], qubits: Tuple, noise: Callable, *noise_args):
    """
    Append noise channel to the end of each program in programs. This noise channel is implemented as a single noisy gate
    acting on the provided qubits.
    
    :param list|program programs: A list of programs (i.e. a Clifford gate) onto each of which will be appended noise.
    :param Tuple qubits: A tuple of the qubits on which each noisy gate should act.
    :param noise: A function which generates the kraus operators of the desired noise.
    :param noise_args: Additional parameters passed on to the noise function.
    """
    for program in programs:
        program.defgate("noise", np.eye(2**len(qubits)))
        program.define_noisy_gate("noise", qubits, noise(*noise_args))
        program.inst(("noise", *qubits))
        
def add_noise_to_sequences(expt: StratifiedExperiment, qubits, noise: Callable, *noise_args):
    """
    Append the given noise to each clifford gate (sequence)
    :param qubits: A tuple of the qubits on which each sequence of Cliffords acts
    :param noise: Function which takes in a gate and appends the desired Krauss operators
    :param noise_args: Additional parameters passed on to the noise function.
    """
    for layer in expt.layers:
        insert_noise(layer.sequence, qubits, noise, *noise_args)
    
def depolarizing_noise(num_qubits: int, p: float =.95):
    """
    Generate the Kraus operators corresponding to a given unitary
    single qubit gate followed by a depolarizing noise channel.

    :params float num_qubits: either 1 or 2 qubit channel supported
    :params float p: parameter in depolarizing channel as defined by: p $\rho$ + (1-p)/d I
    :return: A list, eg. [k0, k1, k2, k3], of the Kraus operators that parametrize the map.
    :rtype: list
    """
    num_of_operators = 4**num_qubits
    probabilities = [p+(1.0-p)/num_of_operators] + [(1.0 - p)/num_of_operators]*(num_of_operators-1)
    return noise.pauli_kraus_map(probabilities)

bm = get_benchmarker()
# establish a connection to a quantum device (in this case virtual)
qc = get_qc("9q-square-qvm")

# Run unitarity RB (This is SLOW)

In [ ]:
single_clifford_p = .95 #p parameter for the depolarizing channel applied to each clifford
num_sequences_per_depth = 50
num_shots = 25
depths = 2 * 2 ** np.arange(4, dtype=np.uint8) # depth = number of cliffords in a sequence
qubits = (0,) # (0,1) for two qubit

expt = generate_unitarity_experiment(bm, qubits, depths, num_sequences_per_depth)

# artificially insert noise on each clifford for simulation purposes
add_noise_to_sequences(expt, qubits, depolarizing_noise, len(qubits), single_clifford_p)

# run num_shots indepedent measurements on the qc 
# for each sequence in the dataframe
# calculate and store purity statistics in the measurement results
results = acquire_unitarity_data(qc, [expt], num_shots)

# fit a model for the first (and only) result StratifiedExperiment
fit = fit_unitarity_results(results[0])

# plot the raw data, point estimate error bars, and fit
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Shifted Purity")

In [ ]:
unitarity = fit.params['decay'].value
print(unitarity)
err = fit.params['decay'].stderr
print(err)

In [ ]:
from forest.benchmarking.randomized_benchmarking import unitarity_to_rb_decay
# Since noise is depolarizing, we expect this value to match the 
# input noise parameter single_clifford_p = .95 
print(unitarity_to_rb_decay(unitarity, 2))
print(unitarity_to_rb_decay(unitarity-err, 2))
print(unitarity_to_rb_decay(unitarity+err, 2))